In [2]:
import joblib
import pandas as pd
import numpy as np

#Load FUOYE's Model
model = joblib.load("../../Models/fuoye_admission_model.pkl")
print("Model Loaded Successfully")

Model Loaded Successfully


In [5]:
# POLICY TABLES (FUOYE RULES)
DEFAULT_UTME_CUTOFF = 150

UTME_CUTOFFS = {
    "Theatre and Media Arts": 200,
    "English and Literary Studies": 200,
    "History and International Studies": 180,
    "Linguistics and Languages": 180,

    "Anatomy": 180,
    "Medical Laboratory Science": 200,
    "Nursing": 220,
    "Physiology": 180,
    "Radiology and Radiography": 200,

    "Agricultural and Bio-Resources Engineering": 160,
    "Civil Engineering": 200,
    "Computer Engineering": 200,
    "Electrical and Electronics Engineering": 200,
    "Mechanical Engineering": 200,
    "Mechatronics Engineering": 220,
    "Metallurgical and Materials Engineering": 160,

    "Architecture": 210,
    "Building": 170,
    "Estate Management": 160,
    "Quantity Surveying": 170,
    "Surveying and Geoinformatics": 160,

    "Accounting": 200,
    "Finance": 160,
    "Business Administration": 200,
    "Public Administration": 160,

    "Biochemistry": 180,
    "Computer Science": 200,
    "Microbiology": 180,

    "Doctor of Pharmacy": 240,
    "Law": 240,
    "Mass Communication": 220,

    "Criminology and Security Studies": 200,
    "Economics": 180,
    "Political Science": 180,
    "Sociology": 160,

    "Medicine and Surgery": 260
}

AGGREGATE_CUTOFFS = {
    "Agricultural Economics and Extension": 57.8,
    "Animal Production and Health": 61.3,
    "Crop Science and Horticulture": 57.3,
    "Fisheries and Aquaculture": 50.3,
    "Food Science and Technology": 58.0,
    "Hospitality and Tourism Management": 58.45,

    "English and Literary Studies": 66.3,
    "History and International Studies": 67.8,
    "Linguistics and Languages": 65.3,
    "Theatre and Media Arts": 65.8,

    "Anatomy": 63.3,
    "Medical Laboratory Science": 72.3,
    "Nursing": 74.6,
    "Physiology": 61.5,
    "Radiology and Radiography": 71.3,

    "Mass Communication": 66.3,
    "Public Relations": 65.0,
    "Journalism": 64.0,
    "Broadcasting": 64.0,
    
    "Computer Science": 65.3,

    "Civil Engineering": 65.0,
    "Computer Engineering": 64.3,
    "Electrical and Electronics Engineering": 63.3,
    "Mechanical Engineering": 65.0,
    "Mechatronics Engineering": 65.0,
    "Agricultural and Bio-Resources Engineering": 55.0,

    "Architecture": 66.25,
    "Building": 57.5,
    "Estate Management": 57.5,
    "Quantity Surveying": 60.5,
    "Surveying andd Geo-informatics": 56.0,
    "Urban and Regional Planning": 56.0,

    "Biology Education": 55.0,
    "Chemistry Education": 55.0,
    "Mathematics Education": 55.5,
    "Physics Education": 53.6,
    "Guidance and Counselling": 59.05,
    "Health Education": 57.95,
    "Human Kinetics": 54.9,
    "Library and Information Science": 64.5,

    "Biochemistry": 63.75,
    "Microbiology": 65.75,
    "Pharmacy": 71.75,
    "Botany": 53.5,
    "Zoology": 55.5,
    "Chemistry": 62.5,
    "Geology": 59.5,
    "Industrial Chemistry": 60.5,
    "Mathematics":55.5,
    "Physics": 56.5,
    "Statistics": 54.5,
    "Demography and Social Statistics": 60.5,
    "Economics": 63.75,
    "Geogrsphy": 58.5,
    "Political Science": 62.5,
    "Psychology": 64.5,
    "Sociology": 63.95
}

# O'LEVEL ENGINE

OLEVEL_SCALE = {
    "A1": 6, "B2": 5, "B3": 4,
    "C4": 3, "C5": 2, "C6": 1
}

def compute_olevel_avg(grades):
    return sum(OLEVEL_SCALE[g] for g in grades) / len(grades)

def compute_aggregate(utme, grades, sittings):
    utme_component = (utme / 400) * 60
    olevel_total = sum(OLEVEL_SCALE[g] for g in grades)
    if sittings == 1:
        olevel_total += 10
    return round(utme_component + olevel_total, 2)

#FEATURE BUILDER (MODEL VIEW)

def build_features(utme, faculty, department, grades, sittings):
    utme_cutoff = UTME_CUTOFFS.get(department, DEFAULT_UTME_CUTOFF)
    aggregate_cutoff = AGGREGATE_CUTOFFS.get(department, 55.0)

    aggregate = compute_aggregate(utme, grades, sittings)

    features = pd.DataFrame([{
        "UTME_Score": utme,
        "Aggregate_Score": aggregate,
        "utme_gap": utme - utme_cutoff,
        "aggregate_gap": aggregate - aggregate_cutoff,
        "Faculty": faculty,
        "Department": department,
        "Olevel_Valid": 1,
        "olevel_avg_points": compute_olevel_avg(grades)
    }])

    return features, aggregate, utme_cutoff, aggregate_cutoff

# EXPLANATION ENGINE

def generate_explanation(utme, aggregate, utme_cutoff, aggregate_cutoff):
    explanation = []

    if utme >= utme_cutoff:
        explanation.append(
            f"Your UTME score is {utme - utme_cutoff} points above the departmental cut-off."
        )
    else:
        explanation.append(
            f"Your UTME score is {utme_cutoff - utme} points below the departmental cut-off."
        )

    if aggregate >= aggregate_cutoff:
        explanation.append(
            f"Your aggregate score is {round(aggregate - aggregate_cutoff, 2)} points above the cut-off."
        )
    else:
        explanation.append(
            f"Your aggregate score is {round(aggregate_cutoff - aggregate, 2)} points below the cut-off."
        )

    return explanation

# INFERENCE FUNCTION

def predict_fuoye_admission(
    utme_score: int,
    faculty: str,
    department: str,
    olevel_grades: list,
    sittings: int
):
    # Hard minimum UTME rule
    if utme_score < DEFAULT_UTME_CUTOFF:
        return {
            "admission_probability": "0.0%",
            "decision": "Not Admitted",
            "explanation": [
                f"FUOYE minimum UTME requirement is {DEFAULT_UTME_CUTOFF}.",
                f"Your UTME score is {utme_score}."
            ]
        }

    X, aggregate, utme_cutoff, aggregate_cutoff = build_features(
        utme_score, faculty, department, olevel_grades, sittings
    )

    probability = model.predict_proba(X)[0][1] * 100
    decision = "Admitted" if probability >= 50 else "Not Admitted"

    explanation = generate_explanation(
        utme_score, aggregate, utme_cutoff, aggregate_cutoff
    )

    return {
        "admission_probability": f"{probability:.1f}%",
        "decision": decision,
        "aggregate_score": aggregate,
        "explanation": explanation
    }

# TEST

if __name__ == "__main__":
    result = predict_fuoye_admission(
        utme_score=235,
        faculty="Engineering",
        department="Mechanical Engineering",
        olevel_grades=["A1", "B3", "B2", "C4", "C5"],
        sittings=1
    )

    print(result)

    if __name__ == "__main__":
        prediction = predict_fuoye_admission(
            utme_score=270,
            faculty="Engineering",
            department="Mechanical Engineering",
            olevel_grades=["A1", "A1", "A1", "B2", "B3"],
            sittings=1
    )

    print(prediction)

{'admission_probability': '18.2%', 'decision': 'Not Admitted', 'aggregate_score': 65.25, 'explanation': ['Your UTME score is 35 points above the departmental cut-off.', 'Your aggregate score is 0.25 points above the cut-off.']}
{'admission_probability': '78.0%', 'decision': 'Admitted', 'aggregate_score': 77.5, 'explanation': ['Your UTME score is 70 points above the departmental cut-off.', 'Your aggregate score is 12.5 points above the cut-off.']}
